In [2]:
# STEP 0 — run this right after you define baseline_2025
coverage_flag_2025 = baseline_2025.calculate(
    "has_marketplace_health_coverage", map_to="household", period=YEAR_FILTER
)

# Convert to Boolean explicitly and keep only the TRUEs
ptc_households = coverage_flag_2025.index[coverage_flag_2025.astype(bool)]

print("Households flagged as Marketplace in 2025:", len(ptc_households))


NameError: name 'baseline_2025' is not defined

In [6]:
# STEP 1 — run this right after you create baseline_2026 (before the reform)

ptc_all = baseline_2026.calculate("aca_ptc", map_to="household", period=YEAR_ANALYZE)
print("Total households with ANY PTC in 2026 baseline:", (ptc_all > 0).sum())

# Check inside the Marketplace subset
baseline_ptc_subset = ptc_all.loc[ptc_households]
print("…inside Marketplace subset:", (baseline_ptc_subset > 0).sum())


Total households with ANY PTC in 2026 baseline: 0.0
…inside Marketplace subset: 0


NameError: name 'baseline_2026' is not defined

In [5]:
# ---------------------------------------------------------------------
# 0. Imports & constants
# ---------------------------------------------------------------------
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

DATASET       = "hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
YEAR_ANALYZE  = 2026   # policy year we care about
YEAR_FILTER   = 2025   # year used to flag Marketplace households
OUTPUT_CSV    = "aca_ptc_extension_impacts.csv"

# ---------------------------------------------------------------------
# 1. Reform: keep the ARPA / IRA enhanced PTC schedule past 2025
# ---------------------------------------------------------------------
aca_extension_reform = Reform.from_dict(
    {
        # 0 – 150 % FPL: zero expected contribution
        "gov.aca.ptc_phase_out_rate[0].amount": {"2026-01-01.2100-12-31": 0},
        "gov.aca.ptc_phase_out_rate[1].amount": {"2026-01-01.2100-12-31": 0},

        # 150 – 400 % FPL glide-path 0 → 8.5 %
        "gov.aca.ptc_phase_out_rate[2].amount": {"2026-01-01.2100-12-31": 0},
        "gov.aca.ptc_phase_out_rate[3].amount": {"2026-01-01.2100-12-31": 0.02},
        "gov.aca.ptc_phase_out_rate[4].amount": {"2026-01-01.2100-12-31": 0.04},
        "gov.aca.ptc_phase_out_rate[5].amount": {"2026-01-01.2100-12-31": 0.06},
        "gov.aca.ptc_phase_out_rate[6].amount": {"2026-01-01.2100-12-31": 0.085},

        # Keep subsidies available above 400 % FPL
        "gov.aca.ptc_income_eligibility[2].amount": {"2026-01-01.2100-12-31": True},
    },
    country_id="us",
)

# ---------------------------------------------------------------------
# 2. Identify Marketplace households in 2025
# ---------------------------------------------------------------------
baseline_2025 = Microsimulation(dataset=DATASET)

coverage_flag_2025 = baseline_2025.calculate(
    "has_marketplace_health_coverage", map_to="household", period=YEAR_FILTER
)
ptc_households = coverage_flag_2025.index[coverage_flag_2025.astype(bool)]

print(f"{len(ptc_households):,} households enrolled in Marketplace coverage in {YEAR_FILTER}")

# ---------------------------------------------------------------------
# 3. Run baseline and reform for 2026
# ---------------------------------------------------------------------
baseline_2026 = Microsimulation(dataset=DATASET)
reform_2026   = Microsimulation(reform=aca_extension_reform, dataset=DATASET)

def pull(sim, var):
    """Convenience: grab a household variable for YEAR_ANALYZE, keep only PTC households."""
    return sim.calculate(var, map_to="household", period=YEAR_ANALYZE).loc[ptc_households]

state               = pull(baseline_2026, "state_code")
employment_income   = pull(baseline_2026, "employment_income")

baseline_ptc        = pull(baseline_2026, "aca_ptc")
reform_ptc          = pull(reform_2026,   "aca_ptc")

baseline_income_hb  = pull(baseline_2026, "household_net_income_including_health_benefits")
reform_income_hb    = pull(reform_2026,   "household_net_income_including_health_benefits")

# ---------------------------------------------------------------------
# 4. Assemble results
# ---------------------------------------------------------------------
df = pd.DataFrame({
    "state"                                  : state,
    "employment_income"                      : employment_income,
    "baseline_aca_ptc"                       : baseline_ptc,
    "reform_aca_ptc"                         : reform_ptc,
    "aca_ptc_change"                         : reform_ptc - baseline_ptc,
    "baseline_net_income_including_hb"       : baseline_income_hb,
    "reform_net_income_including_hb"         : reform_income_hb,
    "net_income_change_including_hb"         : reform_income_hb - baseline_income_hb,
})

df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved {len(df):,} rows to {OUTPUT_CSV}")


/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2,330 households enrolled in Marketplace coverage in 2025
Saved 2,330 rows to aca_ptc_extension_impacts.csv


In [6]:
# Ensure baseline_2026 is defined in a previous cell
baseline_2026.calculate("second_lowest_cost_silver_premium",
                        map_to="household", period=2026).describe()


ValueError: Variable second_lowest_cost_silver_premium does not exist.